# ETL Workspace

The purposes of this notebook is to document and explore prelminary ETL (or possibly ELT) processes necessary for efficient storage, retrieval, and analysis of Degu Lab ephys data.

In [1]:
import os

from degpy.session import Session
from degpy.scraper import Scraper

### Source Data
This is data in original file structure from degu lab. Follows the following structure:
```
data
    080602
        080602_ps01_160614
            2016-06-14_09-39-10
                LFP1.ncs
                LFP2.ncs
                E1.ncs
```         

In [2]:
SOURCE_PATH = '/Volumes/Backup Plus/data'
data = Scraper.crawl_files(SOURCE_PATH) # this is tuple of file paths, data size

In [3]:
print('Number of datafiles: {}'.format(len(data[0])))
print('Total size of files: {:.2f}GB'.format(data[1]))

Number of datafiles: 0
Total size of files: 0.00GB


The above files were all migrated into a single directory for ease of extraction, using the `Scraper.move_files()` function. Need to validate all files were migrated.

In [4]:
DEST_PATH = 'ephys'

print('Numer of datafiles in dest: {}'.format(len(os.listdir(DEST_PATH))))

Numer of datafiles in dest: 2418


This isn't a very robust means of validating migration, but is fine for this exploration phase. Can't compare filenames because I changed filenames to following convention 
    
    '270101_270101_ps10_151015_2015-10-15_09-15-40_LFP4.ncs'

For production migration, will need to write something more robust.

Sorting the list of files and binning them into a dictionary by degu-num and ps.

In [5]:
filenames = os.listdir(DEST_PATH)
sessions = {}

for file in filenames:
    split_file = file.split('_')
    session = split_file[1] + '_' + split_file[2]
    file = os.path.join('ephys', file)
    if session not in sessions:
        sessions[session] = [file]
    else:
        sessions[session].append(file)

In [6]:
sessions

{'270101_ps10': ['ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP4.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP5.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP7.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP6.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_Audio.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP2.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_HC.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP3.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP1.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_CRB.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP12.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP10.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP11.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_R2.ncs',
  'ephys/270101_270101_ps10_151015_2015-10-15_09-15-40_LFP8.nc

## Testing Session module 

In [7]:
dest_files = [os.path.join('ephys', x) for x in os.listdir('ephys')]

def get_lfp_files(lst):
    files = []
    for file in lst:
        if 'LFP' in file:
            files.append(file)
    return files

def get_nev_file(lst):
    for file in lst:
        if 'nev' in file:
            return file
    return None
        

In [8]:
from degpy import neuralynx_io

test_session = sessions['080602_ps17']

test_ncs = neuralynx_io.load_ncs(get_lfp_files(test_session)[0])
test_ncs_data = test_ncs['data']
test_ncs_ts = test_ncs['time']

test_nev = neuralynx_io.load_nev(get_nev_file(test_session))
test_nev_ts = test_nev['events']['TimeStamp']
test_nev_events = test_nev['events']['EventString']




In [9]:
import pandas as pd

ncs_df = pd.DataFrame({'time': test_ncs_ts, 'data': test_ncs_data})

In [10]:
ncs_df.head()

,time,data
0,819292809,-39.307841
1,819293300,-44.251840
2,819293792,-51.332134
3,819294283,-55.421615
4,819294775,-57.618948


In [11]:
print(len(test_ncs_data))
print(len(test_ncs_ts))

14855168
14855168


In [12]:
import numpy as np

test_nev_ts_expanded = np.linspace(test_nev_ts[0], test_nev_ts[1], len(ncs_df))

ncs_df['exposure'] = np.nan * len(ncs_df)

ncs_df['event_ts'] = test_nev_ts_expanded

In [13]:
# Creating mapping for events
# Need them to be float to replace nan's 
# will then apply map to convert to event

event_map = {}
for i in range(len(test_nev_events)):
    event_map[float(i)] = test_nev_events[i].decode('utf-8')

ncs_df['exposure'] = np.nan * len(ncs_df)
event_ixs = []

# Removing last value... need to test this
test_nev_ts = test_nev_ts[:-1]

for i, ts in enumerate(test_nev_ts):
    ## Potential bug: last value from argmax op is 0. delete last value? 
    ix = np.argmax(ncs_df['time'] > ts)
    ncs_df.at[ix, 'exposure'] = float(i)
    
ncs_df['exposure'] = ncs_df['exposure'].fillna(method='ffill').map(event_map)

/Users/maxcopeland/.conda/envs/degpy/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [14]:
ncs_df

,time,data,exposure,event_ts
0,819292809,-39.307841,Starting Recording,8.192923e+08
1,819293300,-44.251840,Starting Recording,8.192923e+08
2,819293792,-51.332134,Starting Recording,8.192923e+08
3,819294283,-55.421615,Starting Recording,8.192923e+08
4,819294775,-57.618948,Starting Recording,8.192923e+08
...,...,...,...,...
14855163,8120653326,7.751702,r2e,8.207046e+08
14855164,8120653326,11.902219,r2e,8.207046e+08
14855165,8120653326,18.555254,r2e,8.207046e+08
14855166,8120653326,19.959106,r2e,8.207046e+08


In [15]:
# Creating ts to exposure lookup
exposure_ts = {exposure.decode('utf-8'):time for exposure, time in zip(test_nev_events, test_nev_ts)}

In [16]:
for root, dirs, files in os.walk(SOURCE_PATH):
    for file in files:
        if 'ncs' in file or 'nev' in file:
            file_path = os.path.join(root, file)
            data_files.append(file_path)
            data_size += os.path.getsize(file_path)


{'Starting Recording': 819292348,
 'r1s': 820704626,
 'r1e': 2357462785,
 'b1s': 2460552980,
 'b1e': 2647660347,
 'b2s': 2681133267,
 'b2e': 2848031726,
 'b3s': 2952754453,
 's1': 4206761289,
 's1o': 4405597635,
 's2': 4224312767,
 's2o': 4425166116,
 'b3e': 3420683097,
 'b4s': 3447851238,
 'b4e': 3872533450,
 'b5s': 3904740176,
 'b5e': 4153336928,
 'b6s': 4184184688,
 'b6e': 4430398038,
 'b7s': 4513503120,
 'o1': 5485778551,
 'o1o': 5656516283,
 'o2': 5488322253,
 'o2o': 5659572495,
 'b7e': 4880694295,
 'b8s': 4893318789,
 'b8e': 5269197231,
 'b9s': 5289374539,
 'b9e': 5473825164,
 'b10s': 5483505847,
 'b10e': 5663204921,
 'b11s': 5673285445,
 'b11e': 5845176089,
 'b12s': 5859240526,
 'b12e': 6043596308,
 'r2s': 6119870140,
 'r2e': 8116978666}

In [23]:
subdirs = [x[0] for x in os.walk(SOURCE_PATH) if '-' in x[0].split('/')[-1]]
for x in subdirs:
    print(x)
# print(subdirs)

In [3]:
os.listdir()

['.DS_Store',
 'degpy',
 'data.json',
 'README.md',
 'sample_data',
 '.gitignore',
 '011520-ETL-exploration.ipynb',
 '.ipynb_checkpoints',
 'ephys',
 '.git',
 'degpy-demo.ipynb',
 '.idea']

In [5]:
sess = Session('sample_data')

In [6]:
term = sess.get_terminal('LFP1.ncs')

In [7]:
df = term.get_dataframe()

In [8]:
df.shape

(14855168, 4)

In [9]:
term.event_timestamps

array([ 819292348,  820704626, 2357462785, 2460552980, 2647660347,
       2681133267, 2848031726, 2952754453, 2959492883, 3178598941,
       3230407375, 3414666506, 3420683097, 3447851238, 3468828139,
       3649999782, 3680383965, 3867330844, 3872533450, 3904740176,
       3921028078, 3941508854, 4124472201, 4147256544, 4153336928,
       4184184688, 4206761289, 4224312767, 4405597635, 4425166116,
       4430398038, 4513503120, 4522351043, 4692722532, 4697587436,
       4876038986, 4880694295, 4893318789, 4894920233, 5077130198,
       5080922159, 5265934075, 5269197231, 5289374539, 5293614543,
       5299486409, 5465105404, 5469121377, 5473825164, 5483505847,
       5485778551, 5488322253, 5656516283, 5659572495, 5663204921,
       5673285445, 5845176089, 5859240526, 6043596308, 6119870140,
       8116978666, 8120860287], dtype=uint64)

In [10]:
df[df.timestamp == 819292348]

,timestamp,data,exposure


In [10]:
df.head()

,timestamp,data,exposure,degu_id
0,8.192928e+08,-41.261026,Starting Recording,080602
1,8.192933e+08,-44.373914,Starting Recording,080602
2,8.192938e+08,-43.763544,Starting Recording,080602
3,8.192943e+08,-35.889768,Starting Recording,080602
4,8.192948e+08,-26.612141,Starting Recording,080602


In [14]:
df[df.timestamp > 820704626]

,timestamp,data,exposure
2873,8.207049e+08,-34.913176,r1s
2874,8.207054e+08,-38.941619,r1s
2875,8.207059e+08,-42.664878,r1s
2876,8.207064e+08,-43.580433,r1s
2877,8.207069e+08,-39.918212,r1s
...,...,...,...
14855163,8.120651e+09,81.606497,r2e
14855164,8.120652e+09,88.137458,r2e
14855165,8.120652e+09,95.156715,r2e
14855166,8.120653e+09,97.231974,r2e


In [16]:
df.to_csv('test-lfp.csv')

In [17]:
term.header

{'FileName': 'C:\\CheetahData\\080602_ps17_160704\\2016-07-04_13-22-47\\LFP1.ncs',
 'TimeOpened': 'Time Opened (m/d/y): 7/4/2016  (h:m:s.ms) 13:22:47.905',
 'TimeOpened_dt': datetime.datetime(2016, 7, 4, 13, 22, 47, 905000),
 'TimeClosed': 'Time Closed (m/d/y): 7/4/2016  (h:m:s.ms) 15:42:17.343',
 'TimeClosed_dt': datetime.datetime(2016, 7, 4, 15, 42, 17, 343000),
 'FileType': 'CSC',
 'FileVersion': '3.3.0',
 'RecordSize': '1044',
 'CheetahRev': '5.6.3',
 'HardwareSubSystemName': 'AcqSystem1',
 'HardwareSubSystemType': 'DigitalLynx',
 'SamplingFrequency': '2034.75',
 'ADMaxValue': '32767',
 'ADBitVolts': '0.000000061037020770982053',
 'AcqEntName': 'LFP1',
 'NumADChannels': '1',
 'ADChannel': '43',
 'InputRange': '2000',
 'InputInverted': 'True',
 'DSPLowCutFilterEnabled': 'True',
 'DspLowCutFrequency': '0.3',
 'DspLowCutNumTaps': '0',
 'DspLowCutFilterType': 'DCO',
 'DSPHighCutFilterEnabled': 'True',
 'DspHighCutFrequency': '500',
 'DspHighCutNumTaps': '64',
 'DspHighCutFilterType': '